In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Loading libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Reading data
data = pd.read_csv(r'../input/customer-churn/churn_modelling.csv',index_col=0)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.nunique()

In [ ]:
data.dtypes

In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(121)
sns.countplot(x='Geography',hue='Exited',data=data)

plt.subplot(122)
sns.countplot(x='Gender',hue='Exited',data=data)

plt.show()

In [ ]:
sns.countplot(x='Exited',data=data)
plt.show()

In [ ]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_mod = ['Gender','Geography']
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [ ]:
data.dtypes

In [ ]:
X = data.drop('Exited',axis=1)
y=data['Exited']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=data['Exited'],test_size=0.3)

In [ ]:
X_train.shape, X_test.shape

### Feature Selection

In [ ]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

In [ ]:
#let's plot the ordered mutual_info values per feature
mutual_info.sort_values(ascending=False).plot.bar(figsize=(14, 7))
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest
#we Will select the  top 5 important features
sel_five_cols = SelectKBest(mutual_info_classif, k=5)
sel_five_cols.fit(X_train, y_train)
X_train.columns[sel_five_cols.get_support()]

In [ ]:
X_train.drop(['Gender','CreditScore','EstimatedSalary','Balance','Tenure'],axis=1)
X_test.drop(['Gender','CreditScore','EstimatedSalary','Balance','Tenure'],axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
model = Sequential()
# Adding the input layer and the first hidden layer
model.add(Dense(units = 6, kernel_initializer='he_uniform', activation='relu', input_dim=X_train.shape[1]))
# Adding the second hidden layer
model.add(Dense(units = 6, kernel_initializer='he_uniform',activation='relu'))
# Adding the output layer
model.add(Dense(units=1, kernel_initializer='glorot_uniform',activation='sigmoid'))

In [ ]:
model.compile(optimizer='Adamax',loss='binary_crossentropy',metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model_history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=10)

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred.round())

In [ ]:
# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()